In [ ]:
#%pip install gcsfs ffspec

In [1]:
import duckdb, os
from fsspec import filesystem

In [3]:
cn = duckdb.connect()
cn.register_filesystem(filesystem('gcs'))
bucket = os.getenv('GCS_BUCKET')
gs_path = f"gs://{bucket}/test_data/data*.csv"
sql = f"select latitude, longitude, email, phonenumber, firstname, txnkey from read_csv_auto('{gs_path}') limit 5"
cn.sql(sql).show(max_width = 1000)

┌────────────┬─────────────┬─────────────────────┬──────────────┬───────────┬──────────────────────────────────────┐
│  Latitude  │  Longitude  │        Email        │ PhoneNumber  │ FirstName │                TxnKey                │
│   double   │   double    │       varchar       │   varchar    │  varchar  │               varchar                │
├────────────┼─────────────┼─────────────────────┼──────────────┼───────────┼──────────────────────────────────────┤
│  63.305305 │ -154.303574 │ WMUVtRc@RfjZciH.com │ 410-815-2637 │ Bertrand  │ 6b691a43-73d9-4784-9397-5a8317418b0f │
│ -48.165188 │  -37.616329 │ MxnNAYJ@ExcgPvo.ru  │ 927-108-4615 │ Ivah      │ c76a1661-006e-4824-b8aa-5c6cdb5b888b │
│ -10.703768 │   80.755714 │ esLuYnd@gnCGTpb.ru  │ 794-536-1081 │ Milo      │ 438be42c-17a8-4cbd-b099-3819c6c95aa6 │
│  -86.62632 │ -160.920425 │ nbTewVT@BeugcbE.net │ 326-711-0895 │ Serenity  │ babd20ec-0e09-4c24-8cd5-5c4ce92b846a │
│ -86.868187 │  -13.870378 │ lDvdkXc@DicIGal.ru  │ 871-069-3124 

In [11]:
b_sql = f"""
select Date as rpt_dt 
    ,count(*) as row_cnt
    ,sum(NetWorth) as tot_net_worth
    ,count(distinct firstName) as fName_cnt
from read_csv_auto('{gs_path}')
group by all
limit 5
"""
cn.sql(b_sql).show()

┌────────────┬─────────┬────────────────────┬───────────┐
│   rpt_dt   │ row_cnt │   tot_net_worth    │ fName_cnt │
│    date    │  int64  │       double       │   int64   │
├────────────┼─────────┼────────────────────┼───────────┤
│ 2018-05-23 │       7 │          218364.03 │         7 │
│ 1980-04-29 │       8 │ 16767798.470000003 │         8 │
│ 1988-10-28 │       8 │  7925525.859999999 │         8 │
│ 1982-01-28 │       6 │ 1612222.1700000002 │         6 │
│ 1972-05-19 │      10 │         1576071.71 │        10 │
└────────────┴─────────┴────────────────────┴───────────┘



In [12]:
b_sql = f"""
select Date as rpt_dt 
    ,count(*) as row_cnt
    ,sum(NetWorth) as tot_net_worth
    ,count(distinct firstName) as fName_cnt
from read_csv_auto('{gs_path}')
group by all
"""

gs_write_path = f"gs://{bucket}/test_data/data_tsfm.parquet"
sql = f"""
COPY ({b_sql}) to '{gs_write_path}' (FORMAT PARQUET)
"""

cn.execute(sql)

In [4]:
sql = f"""
select *
from read_parquet('gs://{bucket}/test_data/data_tsfm.parquet')
"""
cn.sql(sql).show()

┌────────────┬─────────┬────────────────────┬───────────┐
│   rpt_dt   │ row_cnt │   tot_net_worth    │ fName_cnt │
│    date    │  int64  │       double       │   int64   │
├────────────┼─────────┼────────────────────┼───────────┤
│ 2015-04-01 │       6 │           892983.0 │         6 │
│ 1988-02-21 │      11 │           81258.47 │        11 │
│ 1978-03-13 │       8 │ 1488752.5000000002 │         8 │
│ 1997-10-25 │       6 │           11133.06 │         6 │
│ 1972-06-25 │       6 │          533678.01 │         6 │
│ 1985-02-15 │      12 │  7248005.140000001 │        12 │
│ 1979-10-14 │       5 │         1318613.63 │         5 │
│ 1982-02-28 │       5 │ 2319481.6800000006 │         5 │
│ 1995-05-02 │       9 │         4321988.57 │         9 │
│ 2000-06-14 │       8 │  6842205.899999999 │         8 │
│     ·      │       · │              ·     │         · │
│     ·      │       · │              ·     │         · │
│     ·      │       · │              ·     │         · │
│ 2000-07-27 │